In [17]:
import sys
sys.path.append(str('/home/jupyter/'))

import pandas as pd
from configparser import ConfigParser
from repo.MyConfigParser import MyConfigParser
from repo.LoadData import DataLoader
from repo.BuildModel import GenerateModelFile, CreateModel
from repo.Accuracy import AccuracyCalculator, ProphetParameterSearch, PlotAccuracyCharts, MakeFuture
from repo.FileGenerator import *
from repo.Optimization import SavantPSO

In [18]:
updated_config = ConfigParser()
config_in_path = "/home/jupyter/CA/24 Feb Update/config ups ca.ini"
updated_config.read(config_in_path)

getdict = MyConfigParser()
config_dict=getdict.GetDict(updated_config)
region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]

base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]
breakdown_ls = [config_dict['CLIENTINFO'][i] for i in [f'breakdown{i}' for i in range(1,5)] if config_dict['CLIENTINFO'][i] != None]

small_market_str = ''
pre_forecast_period_budget = float(config_dict['CLIENTINFO']['pre_forecast_period_budget'])
client_budget = float(config_dict['CLIENTINFO']['client_budget'])

save_date = config_dict['CLIENTINFO']['save_date']

In [19]:
# Get forecasters
bdDf = pd.read_csv(cuts_df_in_path)#.drop('Unnamed: 0', axis = 1)
cuts_df = bdDf.reset_index(drop = True)
cuts_df['forecaster_name_list'] =  cuts_df['market'] + ' ' + cuts_df['account_type'] + ' ' + cuts_df['funnel'] + '.pkl' #' 05.19.2021 Final.pkl'
cuts_df['periods'] = periods
forecasters_array = cuts_df.forecaster_name_list

# Get analytical file
analytical_file = pd.read_csv(analytical_file_path).drop('Unnamed: 0', axis = 1)
analytical_file['ds'] = pd.to_datetime(analytical_file['ds'])
analytical_file_check_version = analytical_file.copy()

# Get constrain file
constraint_file = pd.read_csv(constraint_file_name)

In [21]:
today = '01.22.2024' #datetime.date.today().strftime("%m.%d.%Y")# 
fileName = base_path + f'Pos Result {today}.csv' #today
pos_df = pd.read_csv(fileName).drop('Unnamed: 0', axis = 1)

In [22]:
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights = pos_df.pos.to_numpy()

daily_simulated_forecast = resultPSO.ResultGenerator(inputWeights)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  687361.0 difference between planned and forecast X:  -0.0
yhat: 259335.0
analytical_spend_latest_period: 636401.9999999998
client_budget - analytical_spend_latest_period: 0


In [23]:
# %load_ext autoreload
# %autoreload 2
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights_base = np.ones(constraint_file.shape[0])

daily_simulated_forecast_base = resultPSO.ResultGenerator(inputWeights_base)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  687361.0 difference between planned and forecast X:  0.0
yhat: 251864.0
analytical_spend_latest_period: 636401.9999999999
client_budget - analytical_spend_latest_period: 0


In [25]:
forecasted_daily_df = daily_simulated_forecast[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
forecasted_daily_df['month'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
forecasted_daily_df['year'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%Y').astype(str)

forecasted_daily_df = forecasted_daily_df.rename(columns = { 'yhat': 'OptimalConversionQuantity', 'X': 'OptimalSpendValue'})
forecasted_daily_df['OptimalSpendValue'] = forecasted_daily_df['OptimalSpendValue'].round(2)

forecasted_daily_df['variable_splits'] = forecasted_daily_df['market'] + '_' + forecasted_daily_df['account_type']+'_'+forecasted_daily_df['funnel']+'_'+forecasted_daily_df['month']+'_'+forecasted_daily_df['year']
print(forecasted_daily_df.OptimalSpendValue.sum(), forecasted_daily_df.OptimalConversionQuantity.sum())
forecasted_daily_df

687360.6399999999 259335.0


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits
0,Canada,-,Brand,2024-01-01,491.0,1182.27,Jan,2024,Canada_-_Brand_Jan_2024
1,Canada,-,Brand,2024-01-02,973.0,2310.57,Jan,2024,Canada_-_Brand_Jan_2024
2,Canada,-,Brand,2024-01-03,765.0,1786.98,Jan,2024,Canada_-_Brand_Jan_2024
3,Canada,-,Brand,2024-01-04,642.0,1460.16,Jan,2024,Canada_-_Brand_Jan_2024
4,Canada,-,Brand,2024-01-05,640.0,1485.99,Jan,2024,Canada_-_Brand_Jan_2024
...,...,...,...,...,...,...,...,...,...
727,Canada,-,Non_Brand,2024-12-27,102.0,821.96,Dec,2024,Canada_-_Non_Brand_Dec_2024
728,Canada,-,Non_Brand,2024-12-28,79.0,551.92,Dec,2024,Canada_-_Non_Brand_Dec_2024
729,Canada,-,Non_Brand,2024-12-29,78.0,509.13,Dec,2024,Canada_-_Non_Brand_Dec_2024
730,Canada,-,Non_Brand,2024-12-30,101.0,607.49,Dec,2024,Canada_-_Non_Brand_Dec_2024


In [26]:
base_daily_df = daily_simulated_forecast_base[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
base_daily_df['month'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
base_daily_df['year'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%Y').astype(str)
base_daily_df['X'] = base_daily_df['X'].round(2)
base_daily_df = base_daily_df.rename(columns = { 'X': 'BaseSpend', 'yhat': 'BaseConversionQuantity'})

print(base_daily_df.BaseSpend.sum(), base_daily_df.BaseConversionQuantity.sum())
base_daily_df

687360.52 251864.0


,market,account_type,funnel,ds,BaseConversionQuantity,BaseSpend,month,year
0,Canada,-,Brand,2024-01-01,453.0,1091.54,Jan,2024
1,Canada,-,Brand,2024-01-02,898.0,2133.24,Jan,2024
2,Canada,-,Brand,2024-01-03,706.0,1649.84,Jan,2024
3,Canada,-,Brand,2024-01-04,592.0,1348.10,Jan,2024
4,Canada,-,Brand,2024-01-05,591.0,1371.95,Jan,2024
...,...,...,...,...,...,...,...,...
727,Canada,-,Non_Brand,2024-12-27,102.0,1009.19,Dec,2024
728,Canada,-,Non_Brand,2024-12-28,79.0,677.63,Dec,2024
729,Canada,-,Non_Brand,2024-12-29,78.0,625.10,Dec,2024
730,Canada,-,Non_Brand,2024-12-30,101.0,745.86,Dec,2024


In [27]:
forecast_n_base_df = pd.merge(forecasted_daily_df, base_daily_df, how = 'left', on = ['market', 'account_type', 'funnel', 'ds', 'month', 'year'])
forecast_n_base_df['SavedDate'] = save_date

forecast_n_base_df.tail()

,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits,BaseConversionQuantity,BaseSpend,SavedDate
727,Canada,-,Non_Brand,2024-12-27,102.0,821.96,Dec,2024,Canada_-_Non_Brand_Dec_2024,102.0,1009.19,_02_05_24
728,Canada,-,Non_Brand,2024-12-28,79.0,551.92,Dec,2024,Canada_-_Non_Brand_Dec_2024,79.0,677.63,_02_05_24
729,Canada,-,Non_Brand,2024-12-29,78.0,509.13,Dec,2024,Canada_-_Non_Brand_Dec_2024,78.0,625.10,_02_05_24
730,Canada,-,Non_Brand,2024-12-30,101.0,607.49,Dec,2024,Canada_-_Non_Brand_Dec_2024,101.0,745.86,_02_05_24
731,Canada,-,Non_Brand,2024-12-31,103.0,1240.07,Dec,2024,Canada_-_Non_Brand_Dec_2024,103.0,1522.53,_02_05_24


# Large Market

In [28]:
lm_df = forecast_n_base_df[forecast_n_base_df['market'] != 'SM'].copy()
lm_split_df = pd.merge(lm_df, analytical_file[['market', 'funnel', 'ds', 'generic_cost_split', 'competitor_cost_split']], how = 'left', on = ['market', 'funnel', 'ds'])

# helper_df is used to get the investment split for each funnel
lm_generic_helper_df = lm_split_df.loc[lm_split_df['funnel'] == 'Non_Brand', ].copy()
lm_generic_helper_df.loc[:,'funnel'] = 'Generic'
lm_generic_helper_df.loc[:,'OptimalSpendValue'] = lm_generic_helper_df['OptimalSpendValue'] * lm_generic_helper_df['generic_cost_split']
lm_generic_helper_df.loc[:,'OptimalConversionQuantity'] = lm_generic_helper_df['OptimalConversionQuantity'] * lm_generic_helper_df['generic_cost_split']#round(lm_generic_helper_df['OptimalConversionQuantity'] * lm_generic_helper_df['generic_cost_split'], 0)
lm_generic_helper_df.loc[:,'BaseSpend'] = lm_generic_helper_df['BaseSpend'] * lm_generic_helper_df['generic_cost_split']
lm_generic_helper_df.loc[:,'BaseConversionQuantity'] = lm_generic_helper_df['BaseConversionQuantity'] * lm_generic_helper_df['generic_cost_split'] #round(lm_generic_helper_df['BaseConversionQuantity'] * lm_generic_helper_df['generic_cost_split'], 0)

lm_competitor_helper_df = lm_split_df.loc[lm_split_df['funnel'] == 'Non_Brand', ].copy()
lm_competitor_helper_df.loc[:,'funnel'] = 'Competitor'
lm_competitor_helper_df.loc[:,'OptimalSpendValue'] = lm_competitor_helper_df['OptimalSpendValue'] * lm_competitor_helper_df['competitor_cost_split']
lm_competitor_helper_df.loc[:,'OptimalConversionQuantity'] = lm_competitor_helper_df['OptimalConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'] #round(lm_competitor_helper_df['OptimalConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'], 0)
lm_competitor_helper_df.loc[:,'BaseSpend'] = lm_competitor_helper_df['BaseSpend'] * lm_competitor_helper_df['competitor_cost_split']
lm_competitor_helper_df.loc[:,'BaseConversionQuantity'] = lm_competitor_helper_df['BaseConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'] #round(lm_competitor_helper_df['BaseConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'], 0)

lm_funnel_level_daily_df = pd.concat([lm_split_df[(lm_split_df['funnel'] == 'Brand') | (lm_split_df['funnel'] == '-')], lm_generic_helper_df, lm_competitor_helper_df])
print(lm_funnel_level_daily_df.market.unique())
lm_funnel_level_daily_df

['Canada']


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits,BaseConversionQuantity,BaseSpend,SavedDate,generic_cost_split,competitor_cost_split
0,Canada,-,Brand,2024-01-01,491.000000,1182.270000,Jan,2024,Canada_-_Brand_Jan_2024,453.000000,1091.540000,_02_05_24,0.500000,0.500000
1,Canada,-,Brand,2024-01-02,973.000000,2310.570000,Jan,2024,Canada_-_Brand_Jan_2024,898.000000,2133.240000,_02_05_24,0.500000,0.500000
2,Canada,-,Brand,2024-01-03,765.000000,1786.980000,Jan,2024,Canada_-_Brand_Jan_2024,706.000000,1649.840000,_02_05_24,0.500000,0.500000
3,Canada,-,Brand,2024-01-04,642.000000,1460.160000,Jan,2024,Canada_-_Brand_Jan_2024,592.000000,1348.100000,_02_05_24,0.500000,0.500000
4,Canada,-,Brand,2024-01-05,640.000000,1485.990000,Jan,2024,Canada_-_Brand_Jan_2024,591.000000,1371.950000,_02_05_24,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Canada,-,Competitor,2024-12-27,14.514678,116.965535,Dec,2024,Canada_-_Non_Brand_Dec_2024,14.514678,143.608506,_02_05_24,0.857699,0.142301
728,Canada,-,Competitor,2024-12-28,7.169051,50.085352,Dec,2024,Canada_-_Non_Brand_Dec_2024,7.169051,61.493218,_02_05_24,0.909253,0.090747
729,Canada,-,Competitor,2024-12-29,5.431083,35.450348,Dec,2024,Canada_-_Non_Brand_Dec_2024,5.431083,43.525254,_02_05_24,0.930371,0.069629
730,Canada,-,Competitor,2024-12-30,8.482908,51.022594,Dec,2024,Canada_-_Non_Brand_Dec_2024,8.482908,62.644178,_02_05_24,0.916011,0.083989


In [29]:
lm_funnel_level_daily_df = lm_funnel_level_daily_df.rename(columns=lambda x: x[0:1].upper() + x[1:])
lm_funnel_level_daily_df = lm_funnel_level_daily_df.drop(['Variable_splits', 'Generic_cost_split', 'Competitor_cost_split'], axis = 1)
lm_funnel_level_daily_df

,Market,Account_type,Funnel,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
0,Canada,-,Brand,2024-01-01,491.000000,1182.270000,Jan,2024,453.000000,1091.540000,_02_05_24
1,Canada,-,Brand,2024-01-02,973.000000,2310.570000,Jan,2024,898.000000,2133.240000,_02_05_24
2,Canada,-,Brand,2024-01-03,765.000000,1786.980000,Jan,2024,706.000000,1649.840000,_02_05_24
3,Canada,-,Brand,2024-01-04,642.000000,1460.160000,Jan,2024,592.000000,1348.100000,_02_05_24
4,Canada,-,Brand,2024-01-05,640.000000,1485.990000,Jan,2024,591.000000,1371.950000,_02_05_24
...,...,...,...,...,...,...,...,...,...,...,...
727,Canada,-,Competitor,2024-12-27,14.514678,116.965535,Dec,2024,14.514678,143.608506,_02_05_24
728,Canada,-,Competitor,2024-12-28,7.169051,50.085352,Dec,2024,7.169051,61.493218,_02_05_24
729,Canada,-,Competitor,2024-12-29,5.431083,35.450348,Dec,2024,5.431083,43.525254,_02_05_24
730,Canada,-,Competitor,2024-12-30,8.482908,51.022594,Dec,2024,8.482908,62.644178,_02_05_24


In [30]:
LM_SM_three_funnel_df = pd.concat([pd.DataFrame(), lm_funnel_level_daily_df]).reset_index(drop = True)
LM_SM_three_funnel_df.insert(3, 'WeekStart', [ i for i in LM_SM_three_funnel_df.Ds.apply(lambda x: x - pd.offsets.Week(weekday=4) )])
LM_SM_three_funnel_df.insert(0, 'Region', region)
LM_SM_three_funnel_df

,Region,Market,Account_type,Funnel,WeekStart,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
0,Canada,Canada,-,Brand,2023-12-29,2024-01-01,491.000000,1182.270000,Jan,2024,453.000000,1091.540000,_02_05_24
1,Canada,Canada,-,Brand,2023-12-29,2024-01-02,973.000000,2310.570000,Jan,2024,898.000000,2133.240000,_02_05_24
2,Canada,Canada,-,Brand,2023-12-29,2024-01-03,765.000000,1786.980000,Jan,2024,706.000000,1649.840000,_02_05_24
3,Canada,Canada,-,Brand,2023-12-29,2024-01-04,642.000000,1460.160000,Jan,2024,592.000000,1348.100000,_02_05_24
4,Canada,Canada,-,Brand,2023-12-29,2024-01-05,640.000000,1485.990000,Jan,2024,591.000000,1371.950000,_02_05_24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,Canada,Canada,-,Competitor,2024-12-20,2024-12-27,14.514678,116.965535,Dec,2024,14.514678,143.608506,_02_05_24
1094,Canada,Canada,-,Competitor,2024-12-27,2024-12-28,7.169051,50.085352,Dec,2024,7.169051,61.493218,_02_05_24
1095,Canada,Canada,-,Competitor,2024-12-27,2024-12-29,5.431083,35.450348,Dec,2024,5.431083,43.525254,_02_05_24
1096,Canada,Canada,-,Competitor,2024-12-27,2024-12-30,8.482908,51.022594,Dec,2024,8.482908,62.644178,_02_05_24


In [31]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df['Ds'] >= '2024-01-01'].groupby(['Funnel', 'Month']).sum(numeric_only=1)

OptimalConversionQuantity  OptimalSpendValue  \
Funnel     Month                                                 
Brand      Apr                 19662.000000       43207.200000   
           Aug                 20561.000000       47561.370000   
           Dec                 17272.000000       43064.640000   
           Feb                 15067.000000       33068.510000   
           Jan                 17047.000000       38569.540000   
           Jul                 26499.000000       60071.180000   
           Jun                 22407.000000       50651.050000   
           Mar                 13435.000000       29336.320000   
           May                 13725.000000       30411.650000   
           Nov                 17576.000000       60871.600000   
           Oct                 21200.000000       51861.190000   
           Sep                 22291.000000       52222.610000   
Competitor Apr                  1956.863330        9925.996234   
           Aug                  2062.843755        4152.669787   
           Dec                   831.057360        5034.388076   
           Feb                  1852.447159       13062.570879   
           Jan                  1305.639057        5690.862001   
           Jul                  2059.428298        5015.896296   
           Jun                  1909.083517        5375.796793   
           Mar                  2056.741110       17516.587563   
           May                  1894.655448        3544.996925   
           Nov                   710.437886        5120.660227   
           Oct                  1786.508710        4317.864780   
           Sep                  2248.765310        6512.641450   
Generic    Apr                   647.136670        3692.503766   
           Aug                   715.156245        1189.500213   
           Dec                  2344.942640       15098.261924   
           Feb                   736.552841        5214.329121   
           Jan                  1356.360943        6698.217999   
           Jul                   687.571702        1490.963704   
           Jun                   630.916483        2058.403207   
           Mar                   571.258890        4960.162437   
           May                   734.344552        1533.513075   
           Nov                  2053.562114       14421.889773   
           Oct                  1041.491290        3866.035220   
           Sep                   399.234690         969.068550   

                  BaseConversionQuantity     BaseSpend  
Funnel     Month                                        
Brand      Apr              17671.000000  38831.300000  
           Aug              20426.000000  47248.090000  
           Dec              15563.000000  38813.690000  
           Feb              13050.000000  28643.460000  
           Jan              15738.000000  35609.500000  
           Jul              23918.000000  54212.660000  
           Jun              21568.000000  48752.780000  
           Mar              12898.000000  28158.290000  
           May              14338.000000  31767.880000  
           Nov              16715.000000  57883.260000  
           Oct              28194.000000  68966.250000  
           Sep              19192.000000  44966.600000  
Competitor Apr               1956.863330  10816.051540  
           Aug               2062.843755   4469.151259  
           Dec                831.057360   6181.124156  
           Feb               1852.447159  14450.277011  
           Jan               1305.639057   7050.520401  
           Jul               2059.428298   4902.655848  
           Jun               1909.083517   5426.029259  
           Mar               2056.741110  19043.180469  
           May               1894.655448   4081.056327  
           Nov                710.437886   4864.271037  
           Oct               1786.508710   5876.904804  
           Sep               2248.765310   7645.487731  
Generic    Apr             

In [32]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df['Ds'] >= '2024-01-01'].groupby(['Funnel', 'Month']).sum(numeric_only=1)

OptimalConversionQuantity  OptimalSpendValue  \
Funnel     Month                                                 
Brand      Apr                 19662.000000       43207.200000   
           Aug                 20561.000000       47561.370000   
           Dec                 17272.000000       43064.640000   
           Feb                 15067.000000       33068.510000   
           Jan                 17047.000000       38569.540000   
           Jul                 26499.000000       60071.180000   
           Jun                 22407.000000       50651.050000   
           Mar                 13435.000000       29336.320000   
           May                 13725.000000       30411.650000   
           Nov                 17576.000000       60871.600000   
           Oct                 21200.000000       51861.190000   
           Sep                 22291.000000       52222.610000   
Competitor Apr                  1956.863330        9925.996234   
           Aug                  2062.843755        4152.669787   
           Dec                   831.057360        5034.388076   
           Feb                  1852.447159       13062.570879   
           Jan                  1305.639057        5690.862001   
           Jul                  2059.428298        5015.896296   
           Jun                  1909.083517        5375.796793   
           Mar                  2056.741110       17516.587563   
           May                  1894.655448        3544.996925   
           Nov                   710.437886        5120.660227   
           Oct                  1786.508710        4317.864780   
           Sep                  2248.765310        6512.641450   
Generic    Apr                   647.136670        3692.503766   
           Aug                   715.156245        1189.500213   
           Dec                  2344.942640       15098.261924   
           Feb                   736.552841        5214.329121   
           Jan                  1356.360943        6698.217999   
           Jul                   687.571702        1490.963704   
           Jun                   630.916483        2058.403207   
           Mar                   571.258890        4960.162437   
           May                   734.344552        1533.513075   
           Nov                  2053.562114       14421.889773   
           Oct                  1041.491290        3866.035220   
           Sep                   399.234690         969.068550   

                  BaseConversionQuantity     BaseSpend  
Funnel     Month                                        
Brand      Apr              17671.000000  38831.300000  
           Aug              20426.000000  47248.090000  
           Dec              15563.000000  38813.690000  
           Feb              13050.000000  28643.460000  
           Jan              15738.000000  35609.500000  
           Jul              23918.000000  54212.660000  
           Jun              21568.000000  48752.780000  
           Mar              12898.000000  28158.290000  
           May              14338.000000  31767.880000  
           Nov              16715.000000  57883.260000  
           Oct              28194.000000  68966.250000  
           Sep              19192.000000  44966.600000  
Competitor Apr               1956.863330  10816.051540  
           Aug               2062.843755   4469.151259  
           Dec                831.057360   6181.124156  
           Feb               1852.447159  14450.277011  
           Jan               1305.639057   7050.520401  
           Jul               2059.428298   4902.655848  
           Jun               1909.083517   5426.029259  
           Mar               2056.741110  19043.180469  
           May               1894.655448   4081.056327  
           Nov                710.437886   4864.271037  
           Oct               1786.508710   5876.904804  
           Sep               2248.765310   7645.487731  
Generic    Apr             

In [33]:
# Construct a BigQuery client object.
bqclient = bigquery.Client()

target_table_name = 'budget_allocation_scenario_with_base_' + region + save_date
table_id = "ups-analytics.forecast_optimization." + target_table_name

# job_config.
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")

load_job = bqclient.load_table_from_dataframe(
    forecast_n_base_df, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bqclient.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 732 rows.


table = bqclient.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [34]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df.Ds>='2024-01-01'].to_csv(base_path + 'output.csv')